crear tablas

In [1]:
import sqlite3

# Conectar a la base de datos (o crearla si no existe)
conn = sqlite3.connect('olimpicos.db')
cursor = conn.cursor()

# Crear la tabla de equipos con la columna ranking_mundial
cursor.execute('''
    CREATE TABLE IF NOT EXISTS equipos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        equipo TEXT NOT NULL,
        grupo TEXT NOT NULL
    )
''')

# Crear la tabla de partidos con la columna dia_de_la_semana
cursor.execute('''
    CREATE TABLE IF NOT EXISTS partidos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        equipo1 TEXT NOT NULL,
        equipo2 TEXT NOT NULL,
        info_partido TEXT NOT NULL,
        hora TEXT NOT NULL,
        fecha TEXT NOT NULL,
        dia_de_la_semana TEXT NOT NULL,
        FOREIGN KEY (equipo1) REFERENCES equipos(equipo),
        FOREIGN KEY (equipo2) REFERENCES equipos(equipo)
    )
''')

# Crear la tabla de jugadores
cursor.execute('''
    CREATE TABLE IF NOT EXISTS jugadores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        equipo TEXT NOT NULL,
        jugador TEXT NOT NULL,
        fecha_nacimiento TEXT NOT NULL,
        altura REAL NOT NULL,
        club_actual TEXT NOT NULL,
        FOREIGN KEY (equipo) REFERENCES equipos(equipo)
    )
''')

# Confirmar los cambios
conn.commit()

# Cerrar la conexión a la base de datos
conn.close()


borrar tabla

In [8]:
import sqlite3

# Conectar a la base de datos (o crearla si no existe)
conn = sqlite3.connect('olimpicos.db')
cursor = conn.cursor()

# Eliminar la tabla partidos si existe
cursor.execute('DROP TABLE IF EXISTS partidos')

# Confirmar los cambios
conn.commit()

# Cerrar la conexión a la base de datos
conn.close()


In [15]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('olimpicos.db')

# Leer la tabla 'partidos' en un DataFrame de pandas
df = pd.read_sql_query("SELECT * FROM partidos", conn)

# Mostrar las primeras filas del DataFrame
print(df.head())

# Cerrar la conexión a la base de datos
conn.close()


   id      equipo1      equipo2              info_partido   hora        fecha  \
0   1    Australia        Spain  FASE DE GRUPOS · GRUPO A  04:00  27 JUL/2024   
1   2      Germany        Japan  FASE DE GRUPOS · GRUPO B  06:30  27 JUL/2024   
2   3       France       Brazil  FASE DE GRUPOS · GRUPO B  10:15  27 JUL/2024   
3   4       Greece       Canada  FASE DE GRUPOS · GRUPO A  14:00  27 JUL/2024   
4   5  South Sudan  Puerto Rico  FASE DE GRUPOS · GRUPO C  04:00  28 JUL/2024   

  dia_de_la_semana  
0              SÁB  
1              SÁB  
2              SÁB  
3              SÁB  
4              DOM  


In [17]:
df['equipo1'].unique()

array(['Australia', 'Germany', 'France', 'Greece', 'South Sudan',
       'Serbia', 'Spain', 'Canada', 'Japan', 'Brazil', 'Puerto Rico',
       'United States', 'TBD'], dtype=object)

: 

In [13]:
import pandas as pd

# Diccionario para la conversión de abreviaturas a nombres completos
pais_dict = {
    'AUS': 'Australia',
    'GER': 'Alemania',
    'FRA': 'Francia',
    'GRE': 'Grecia',
    'SSD': 'sudan del sur',
    'SRB': 'Serbia',
    'ESP': 'España',
    'CAN': 'Canada',
    'JPN': 'Japon',
    'BRA': 'Brazil',
    'PUR': 'Puerto Rico',
    'USA': 'Estados Unidos'
}

def convertir_abreviaturas(df):
    """
    Convierte las abreviaturas de países en nombres completos en las columnas 'equipo1' y 'equipo2' de un DataFrame.

    Args:
        df (pd.DataFrame): DataFrame que contiene las columnas 'equipo1' y 'equipo2' con abreviaturas de países.

    Returns:
        pd.DataFrame: DataFrame con las abreviaturas reemplazadas por nombres completos.
    """
    # Reemplazar abreviaturas en las columnas 'equipo1' y 'equipo2'
    df['equipo1'] = df['equipo1'].map(pais_dict).fillna(df['equipo1'])
    df['equipo2'] = df['equipo2'].map(pais_dict).fillna(df['equipo2'])
    
    return df

paices = pd.DataFrame(df)

# Aplicar la función de conversión
df_convertido = convertir_abreviaturas(df)

print(df_convertido)


    id        equipo1        equipo2              info_partido         hora  \
0    1      Australia          Spain  FASE DE GRUPOS · GRUPO A        04:00   
1    2        Germany          Japan  FASE DE GRUPOS · GRUPO B        06:30   
2    3         France         Brazil  FASE DE GRUPOS · GRUPO B        10:15   
3    4         Greece         Canada  FASE DE GRUPOS · GRUPO A        14:00   
4    5    South Sudan    Puerto Rico  FASE DE GRUPOS · GRUPO C        04:00   
5    6         Serbia  United States  FASE DE GRUPOS · GRUPO C        10:15   
6    7          Spain         Greece  FASE DE GRUPOS · GRUPO A        04:00   
7    8         Canada      Australia  FASE DE GRUPOS · GRUPO A        06:30   
8    9          Japan         France  FASE DE GRUPOS · GRUPO B        10:15   
9   10         Brazil        Germany  FASE DE GRUPOS · GRUPO B        14:00   
10  11    Puerto Rico         Serbia  FASE DE GRUPOS · GRUPO C        10:15   
11  12  United States    South Sudan  FASE DE GRUPOS

añadir datos

In [11]:
import re
import sqlite3
import pandas as pd

def insertar_datos_en_sqlite(db_name, table_name, dataframe):
    """
    Inserta los datos de un DataFrame en una tabla SQLite.

    :param db_name: Nombre de la base de datos SQLite.
    :param table_name: Nombre de la tabla en la base de datos.
    :param dataframe: DataFrame con los datos a insertar.
    """
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Iniciar una transacción
    cursor.execute('BEGIN TRANSACTION')

    # Función para encerrar entre comillas los nombres de columnas que lo necesiten
    def quote_identifier(s):
        return f'"{s}"' if re.match(r'^[0-9]', s) or ' ' in s or s.upper() in ['TO', 'FROM', 'WHERE', 'GROUP', 'ORDER'] else s

    # Crear una lista de nombres de columnas con los identificadores apropiadamente citados
    columnas = [quote_identifier(col) for col in dataframe.columns]

    # Crear la parte de los valores de la consulta SQL
    valores = ', '.join(['?' for _ in dataframe.columns])

    # Crear la consulta SQL
    sql = f"INSERT INTO {table_name} ({', '.join(columnas)}) VALUES ({valores})"

    # Iterar sobre las filas del DataFrame
    for _, fila in dataframe.iterrows():
        # Ejecutar la consulta SQL con los valores de la fila
        cursor.execute(sql, tuple(fila))

    # Confirmar la transacción
    conn.commit()

    # Cerrar la conexión
    conn.close()

In [12]:
insertar_datos_en_sqlite('olimpicos.db', 'partidos', df_convertido)